In [1]:
import math
from collections import Counter
import csv
import statistics

class Node:
    def __init__(self, attribute=None, threshold=None, label=None, branches=None):
        self.attribute = attribute
        self.threshold = threshold
        self.label = label
        self.branches = branches or {}

def id3(data, attributes, label_index, max_depth, criterion='info_gain'):
    labels = [row[label_index] for row in data]
    
    if not labels:  # Handle empty dataset
        return Node(label=None)
    
    if len(set(labels)) == 1:
        return Node(label=labels[0])
    if len(attributes) == 0 or max_depth == 0:
        return Node(label=max(set(labels), key=labels.count))
    
    best_attribute, threshold = choose_best_attribute(data, attributes, label_index, criterion)
    node = Node(attribute=best_attribute, threshold=threshold)
    
    if threshold is not None:
        # Binary split for numerical attributes
        left_subset = [row for row in data if row[best_attribute] <= threshold]
        right_subset = [row for row in data if row[best_attribute] > threshold]
        if left_subset:
            node.branches[f"<={threshold}"] = id3(left_subset, attributes, label_index, max_depth - 1, criterion)
        if right_subset:
            node.branches[f">{threshold}"] = id3(right_subset, attributes, label_index, max_depth - 1, criterion)
    else:
        # Categorical split
        for value in set(row[best_attribute] for row in data):
            subset = [row for row in data if row[best_attribute] == value]
            if subset:
                node.branches[value] = id3(subset, attributes, label_index, max_depth - 1, criterion)
    
    if not node.branches:  # If no branches were created, make this a leaf node
        return Node(label=max(set(labels), key=labels.count))
    
    return node

def choose_best_attribute(data, attributes, label_index, criterion):
    best_gain = -float('inf')
    best_attribute = None
    best_threshold = None
    
    for attribute in attributes:
        if all(isinstance(row[attribute], (int, float)) for row in data):
            # Numerical attribute
            values = [row[attribute] for row in data]
            threshold = statistics.median(values)
            gain = calculate_gain(data, attribute, label_index, criterion, threshold)
        else:
            # Categorical attribute
            gain = calculate_gain(data, attribute, label_index, criterion)
            threshold = None
        
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute
            best_threshold = threshold
    
    return best_attribute, best_threshold

def calculate_gain(data, attribute, label_index, criterion, threshold=None):
    if criterion == 'info_gain':
        return information_gain(data, attribute, label_index, threshold)
    elif criterion == 'majority_error':
        return majority_error(data, attribute, label_index, threshold)
    elif criterion == 'gini_index':
        return gini_index(data, attribute, label_index, threshold)
    else:
        raise ValueError("Invalid criterion. Choose 'info_gain', 'majority_error', or 'gini_index'.")

def entropy(data, label_index):
    labels = [row[label_index] for row in data]
    counts = Counter(labels)
    total = len(labels)
    return -sum((count / total) * math.log2(count / total) for count in counts.values())

def information_gain(data, attribute, label_index, threshold=None):
    total_entropy = entropy(data, label_index)
    weighted_entropy = 0
    
    if threshold is not None:
        # Binary split for numerical attributes
        left_subset = [row for row in data if row[attribute] <= threshold]
        right_subset = [row for row in data if row[attribute] > threshold]
        weighted_entropy = (len(left_subset) / len(data)) * entropy(left_subset, label_index) + \
                           (len(right_subset) / len(data)) * entropy(right_subset, label_index)
    else:
        # Categorical split
        for value in set(row[attribute] for row in data):
            subset = [row for row in data if row[attribute] == value]
            weight = len(subset) / len(data)
            weighted_entropy += weight * entropy(subset, label_index)
    
    return total_entropy - weighted_entropy

def majority_error(data, attribute, label_index, threshold=None):
    total_error = 1 - max(Counter(row[label_index] for row in data).values()) / len(data)
    weighted_error = 0
    
    if threshold is not None:
        # Binary split for numerical attributes
        left_subset = [row for row in data if row[attribute] <= threshold]
        right_subset = [row for row in data if row[attribute] > threshold]
        left_error = 1 - max(Counter(row[label_index] for row in left_subset).values()) / len(left_subset) if left_subset else 0
        right_error = 1 - max(Counter(row[label_index] for row in right_subset).values()) / len(right_subset) if right_subset else 0
        weighted_error = (len(left_subset) / len(data)) * left_error + (len(right_subset) / len(data)) * right_error
    else:
        # Categorical split
        for value in set(row[attribute] for row in data):
            subset = [row for row in data if row[attribute] == value]
            weight = len(subset) / len(data)
            majority_class = max(set(row[label_index] for row in subset), key=lambda c: sum(1 for row in subset if row[label_index] == c))
            error = sum(1 for row in subset if row[label_index] != majority_class) / len(subset)
            weighted_error += weight * error
    
    return total_error - weighted_error

def gini_index(data, attribute, label_index, threshold=None):
    def calculate_gini(subset):
        counts = Counter(row[label_index] for row in subset)
        total = len(subset)
        return 1 - sum((count / total) ** 2 for count in counts.values())

    total_gini = calculate_gini(data)
    weighted_gini = 0
    
    if threshold is not None:
        # Binary split for numerical attributes
        left_subset = [row for row in data if row[attribute] <= threshold]
        right_subset = [row for row in data if row[attribute] > threshold]
        weighted_gini = (len(left_subset) / len(data)) * calculate_gini(left_subset) + \
                        (len(right_subset) / len(data)) * calculate_gini(right_subset)
    else:
        # Categorical split
        for value in set(row[attribute] for row in data):
            subset = [row for row in data if row[attribute] == value]
            weight = len(subset) / len(data)
            weighted_gini += weight * calculate_gini(subset)
    
    return total_gini - weighted_gini

def predict(node, instance):
    if node.label is not None:
        return node.label
    if node.attribute >= len(instance):  # Handle case where attribute is out of range
        return None
    value = instance[node.attribute]
    if node.threshold is not None:
        if value <= node.threshold:
            branch = f"<={node.threshold}"
        else:
            branch = f">{node.threshold}"
    else:
        branch = value
    if branch not in node.branches:
        return None  # Return None if we can't make a prediction
    return predict(node.branches[branch], instance)

def load_data(filename):
    data = []
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            instance = []
            for value in row:
                try:
                    # Try to convert to float (for numerical attributes)
                    instance.append(float(value))
                except ValueError:
                    # If conversion fails, treat as categorical
                    instance.append(value)
            data.append(instance)
    return data

def calculate_error(tree, data, label_index):
    incorrect = sum(1 for instance in data if predict(tree, instance) != instance[label_index])
    return incorrect / len(data)

def run_experiment(train_data, test_data, max_depths, criteria):
    num_attributes = len(train_data[0]) - 1
    attributes = list(range(num_attributes))
    label_index = -1  # Assume the last column is the label
    
    results = {criterion: {depth: {'train': 0, 'test': 0} for depth in max_depths} for criterion in criteria}
    
    for criterion in criteria:
        for depth in max_depths:
            tree = id3(train_data, attributes, label_index, depth, criterion)
            train_error = calculate_error(tree, train_data, label_index)
            test_error = calculate_error(tree, test_data, label_index)
            results[criterion][depth]['train'] = train_error
            results[criterion][depth]['test'] = test_error
    
    return results

In [2]:
# Load data
train_data = load_data('train_bank.csv')
test_data = load_data('test_bank.csv')

# Run experiment
max_depths = range(1, 17)
criteria = ['info_gain', 'majority_error', 'gini_index']
results = run_experiment(train_data, test_data, max_depths, criteria)

# Print results
print("Depth | Information Gain | Majority Error | Gini Index")
print("      | Train  | Test    | Train  | Test   | Train | Test")
print("------|--------|---------|--------|--------|-------|------")
for depth in max_depths:
    print(f"{depth:5d} | {results['info_gain'][depth]['train']:.4f} | {results['info_gain'][depth]['test']:.4f} | "
          f"{results['majority_error'][depth]['train']:.4f} | {results['majority_error'][depth]['test']:.4f} | "
          f"{results['gini_index'][depth]['train']:.4f} | {results['gini_index'][depth]['test']:.4f}")

Depth | Information Gain | Majority Error | Gini Index
      | Train  | Test    | Train  | Test   | Train | Test
------|--------|---------|--------|--------|-------|------
    1 | 0.1192 | 0.1248 | 0.1088 | 0.1166 | 0.1088 | 0.1166
    2 | 0.1060 | 0.1114 | 0.1066 | 0.1134 | 0.1070 | 0.1142
    3 | 0.1034 | 0.1136 | 0.1000 | 0.1158 | 0.0922 | 0.1272
    4 | 0.0886 | 0.1268 | 0.0894 | 0.1300 | 0.0748 | 0.1328
    5 | 0.0728 | 0.1358 | 0.0718 | 0.1384 | 0.0636 | 0.1400
    6 | 0.0552 | 0.1394 | 0.0646 | 0.1438 | 0.0482 | 0.1530
    7 | 0.0376 | 0.1476 | 0.0600 | 0.1476 | 0.0304 | 0.1618
    8 | 0.0198 | 0.1556 | 0.0574 | 0.1476 | 0.0178 | 0.1688
    9 | 0.0096 | 0.1590 | 0.0566 | 0.1492 | 0.0082 | 0.1696
   10 | 0.0024 | 0.1612 | 0.0562 | 0.1492 | 0.0028 | 0.1714
   11 | 0.0010 | 0.1616 | 0.0562 | 0.1492 | 0.0012 | 0.1730
   12 | 0.0000 | 0.1622 | 0.0562 | 0.1492 | 0.0000 | 0.1730
   13 | 0.0000 | 0.1622 | 0.0562 | 0.1492 | 0.0000 | 0.1730
   14 | 0.0000 | 0.1622 | 0.0562 | 0.1492 | 0.00